# 0. Import libraries

In [1]:
import networkx as nx
import pandas as pd
import statistics
import collections
import timeit

# 1. Reading data

### Reading reduced edges file

In [2]:
reduced = pd.read_csv('wiki-topcats-reduced.txt', names=['source', 'destination'], sep='\t')

In [3]:
reduced.head()

,source,destination
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


In [4]:
# Number of edges
print('Number of edges:', len(reduced)) 

Number of edges: 2645247


In [5]:
# Number of nodes
set_nodes = set([*list(reduced['source']), *list(reduced['destination'])])
print('Number of nodes:', len(set_nodes)) 

Number of nodes: 461193


### Reading page names file

In [6]:
#The ID is the same as the index of row so we are only saving the names ordered
def read_names_list(file_path):
    names_list = []
    with open(file_path) as file:
        for line in file:
            cline = line.rstrip().split()
            #row_id = cline[0]
            row_name = cline[1:]
            #names_list.append((row_id, " ".join(row_name)))
            names_list.append(" ".join(row_name))
    return names_list
        
names_list = read_names_list('wiki-topcats-page-names.txt')

In [7]:
names_list[0]

'Chiasmal syndrome'

In [8]:
len(names_list)

1791489

### Reading categories file

In [9]:
def read_categories_list(file_path):
    categories = {}
    with open(file_path) as file:
        for line in file:
            row_cat = line.rstrip().partition(':')[2].partition('; ')[0]
            row_artic = list(map(int, line.rstrip().partition(':')[2].partition('; ')[2].split()))
            if len(row_artic) > 3500: # Each category must have more than 3500 nodes
                categories[row_cat] = row_artic
    return categories

categories = read_categories_list('wiki-topcats-categories.txt')

In [10]:
# Print categories keys and length of the contained list
length_categories = {key: len(value) for key, value in categories.items()}
print(length_categories)
del length_categories

{'English_footballers': 9237, 'The_Football_League_players': 9467, 'Association_football_forwards': 6959, 'Association_football_goalkeepers': 3997, 'Association_football_midfielders': 8270, 'Association_football_defenders': 6668, 'Living_people': 418223, 'Year_of_birth_unknown': 3760, 'Harvard_University_alumni': 6154, 'Major_League_Baseball_pitchers': 6580, 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 6546, 'Indian_films': 5913, 'Year_of_death_missing': 7851, 'English_cricketers': 3813, 'Year_of_birth_missing_(living_people)': 34721, 'Rivers_of_Romania': 7729, 'Main_Belt_asteroids': 13704, 'Asteroids_named_for_people': 5701, 'English-language_albums': 4853, 'English_television_actors': 3501, 'British_films': 4551, 'English-language_films': 22699, 'American_films': 15302, 'Fellows_of_the_Royal_Society': 3697, 'People_from_New_York_City': 4888, 'American_Jews': 3542, 'American_television_actors': 11661, 'American_film_actors': 13938, 'Debut_albums': 8401, 'Blac

In [11]:
print("Number of categories:", len(categories.keys()))

Number of categories: 35


Now we have to intersect the dictionary of categories with the set of nodes in reduced file:

In [12]:
# Intersect categories and set_nodes of reduced
def intersect_categories_reduced(categories, set_nodes):
    categories_reduced = {}
    for key, values in categories.items():
        aux_list = set_nodes.intersection(values)
        if(len(aux_list) > 3500): # Each category must have more than 3500 nodes
            categories_reduced[key] = aux_list   
    return categories_reduced

categories_reduced = intersect_categories_reduced(categories, set_nodes)

In [13]:
length_categories_reduced = {key: len(value) for key, value in categories_reduced.items()}
print(length_categories_reduced)
del length_categories_reduced

{'English_footballers': 7538, 'The_Football_League_players': 7814, 'Association_football_forwards': 5097, 'Association_football_goalkeepers': 3737, 'Association_football_midfielders': 5827, 'Association_football_defenders': 4588, 'Living_people': 348300, 'Harvard_University_alumni': 5549, 'Major_League_Baseball_pitchers': 5192, 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 6491, 'Indian_films': 5568, 'Year_of_death_missing': 4122, 'Year_of_birth_missing_(living_people)': 28498, 'Rivers_of_Romania': 7729, 'Main_Belt_asteroids': 11660, 'Asteroids_named_for_people': 4895, 'English-language_albums': 4760, 'British_films': 4422, 'English-language_films': 22463, 'American_films': 15159, 'People_from_New_York_City': 4614, 'American_television_actors': 11531, 'American_film_actors': 13865, 'Debut_albums': 7561, 'Black-and-white_films': 10759, 'Year_of_birth_missing': 4346, 'Place_of_birth_missing_(living_people)': 5532, 'American_military_personnel_of_World_War_II': 37

In [14]:
print("Number of categories_reduced:", len(categories_reduced.keys()))

Number of categories_reduced: 29


# [RQ1] Build the graph

Build the graph G=(V, E), where V is the set of articles and E the hyperlinks among them. For this section we are using networkx to build the graph.

We have a file with the source and destination node id so it seems to be a directed graph. Just to be sure we are going to build to graphs. Directed and Undirected graph.

In [15]:
# Create a directed Graph
Gdir = nx.DiGraph() # Directed graph
#Gund = nx.Graph() # Undirected graph

# Insert Nodes to both graphs
Gdir.add_nodes_from(set_nodes)
#Gund.add_nodes_from(set_nodes)

# Add Edges to both graphs
for index, row in reduced.iterrows():
    Gdir.add_edge(row['source'], row['destination'])
    #Gund.add_edge(row['source'], row['destination'])

In [16]:
print("Edges in file wiki-topcats-reduced.txt: ", len(reduced))
print("Edges in graph directed: ", Gdir.number_of_edges(), )
#print("Edges in graph undirected: ", Gund.number_of_edges())

Edges in file wiki-topcats-reduced.txt:  2645247
Edges in graph directed:  2645247


We can observe that it is a directed graph because there are more edges in the directed so some of them are unidirected. (To be undirected need to have all source/destination in both directions)

In [17]:
# Directed graph
print("Directed Graph:")
print(nx.info(Gdir),'\n')
# Undirected graph
#print("Undirected Graph:")
#print(nx.info(Gund))

Directed Graph:
Name: 
Type: DiGraph
Number of nodes: 461193
Number of edges: 2645247
Average in degree:   5.7357
Average out degree:   5.7357 



In [18]:
nx.density(Gdir)

1.2436602635647606e-05

### Graph results:

- It is a **DIRECTED graph** because the edges given are source-destination
- The number of nodes: **461193**
- The number of edges: **2645247**
- The average node degree: **Average in degree:   5.7357 | Average out degree:   5.7357** Each node has on average 5.7 nodes in and 5.7 nodes out
- Is the graph dense?: **density = 1.24e-05** The density is 0 for a graph without edges and 1 for a complete graph. So it is not dense.

### Clean active variables to save memory

In [19]:
%whos

Variable                       Type         Data/Info
-----------------------------------------------------
Gdir                           DiGraph      
categories                     dict         n=35
categories_reduced             dict         n=29
collections                    module       <module 'collections' fro<...>ollections\\__init__.py'>
index                          int          2645246
intersect_categories_reduced   function     <function intersect_categ<...>ed at 0x000001E22B0A12F0>
names_list                     list         n=1791489
nx                             module       <module 'networkx' from '<...>\\networkx\\__init__.py'>
pd                             module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
read_categories_list           function     <function read_categories<...>st at 0x000001E22B097950>
read_names_list                function     <function read_names_list at 0x000001E2181D36A8>
reduced                        DataFrame          

In [20]:
#set_nodes, reduced, names_list, categories, categories_reduced, Gdir, Gund, index, row
del set_nodes, reduced, names_list, categories, index, row # Remove unused variables

In [21]:
%whos

Variable                       Type        Data/Info
----------------------------------------------------
Gdir                           DiGraph     
categories_reduced             dict        n=29
collections                    module      <module 'collections' fro<...>ollections\\__init__.py'>
intersect_categories_reduced   function    <function intersect_categ<...>ed at 0x000001E22B0A12F0>
nx                             module      <module 'networkx' from '<...>\\networkx\\__init__.py'>
pd                             module      <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
read_categories_list           function    <function read_categories<...>st at 0x000001E22B097950>
read_names_list                function    <function read_names_list at 0x000001E2181D36A8>
statistics                     module      <module 'statistics' from<...>da3\\lib\\statistics.py'>
timeit                         module      <module 'timeit' from 'C:<...>aconda3\\lib\\timeit.py'>


# [RQ2] Block Ranking

Obtain a block-ranking, where the blocks are represented by the categories. In particular, we want: block_ranking = [C0, C1, C2...]

Each category  corresponds to a list of nodes.
The order of the remaining categories is given by:

**distance(Co, Ci) = median(ShortestPath(Co, Ci))**

## TEST ALGORITHM COMPARISON

In this section we are testing the **networkx method (Test 2) vs our own implementation (Test 2)** to compute the shortest_path_length between 2 categories

### Test 1 (networkX functions)

In [46]:
# USING shortest_path_length
def distance_time_networkx():
    CxCy = []
    # Input category to make block ranking
    source_category = 'English_footballers'
    target_category = 'The_Football_League_players'

    C0 = list(categories_reduced.get(source_category)) #C0
    CI = list(categories_reduced.get(target_category)) #C1
    
    for s_value in C0[:10]:
        for t_value in CI:
            #if s_value != t_value:
                try:
                    CxCy.append(nx.shortest_path_length(Gdir, source=s_value, target=t_value))
                except:
                    pass

    print("#Results:", len(CxCy), "Median:", statistics.median(CxCy))
    return statistics.median(CxCy)

In [47]:
timeit.timeit(distance_time_networkx, number = 1)

#Results: 48967 Median: 6


9.715925347812572

### Test 2 (Own implementation)

In [24]:
dic_adj = dict(Gdir.adjacency())

In [25]:
def shortest_path_bfs_list(dic_adj_graph, source, destination_list): 
    visited = {source}
    queue = collections.deque([(source, 0)])
    jumps_list = [] # To store the number of jumps of each node in desination_list
    
    # Repeat until queue is empty
    while queue: 
        vertex = queue.popleft()
        # If vertex is destionation return level
        if vertex[0] in destination_list:
            jumps_list.append(vertex[1])
        # Add to visited and append to queue
        for neighbour in dic_adj_graph[vertex[0]]: 
            if neighbour not in visited: 
                visited.add(neighbour) 
                queue.append((neighbour, vertex[1]+1)) 
    return jumps_list

In [48]:
# THIS FUNCTION IS ONLY FOR TIMING
def distance_time_our():
    CxCy = []
    # Input category to make block ranking
    source_category = 'English_footballers'
    target_category = 'The_Football_League_players'

    C0 = list(categories_reduced.get(source_category)) #C0
    CI = set(categories_reduced.get(target_category)) #C1
    
    for s_value in C0[:10]:
        CxCy.extend(shortest_path_bfs_list(dic_adj, s_value, CI))

    print("#Results:", len(CxCy), "Median:", statistics.median(CxCy))
    return statistics.median(CxCy)

In [49]:
timeit.timeit(distance_time_our, number = 1)

#Results: 48967 Median: 6


9.537123952367892

### Test 3 (Own implementation)

In [28]:
def shortest_path_bfs_list_list(dic_adj_graph, source_list, destination_list): 
    visited = set()
    queue = collections.deque([])
    for node in source_list:
        visited.add((node, node))
        queue.append((node, 0, node))
    jumps_list = [] # To store the number of jumps of each node in desination_list
    
    # Repeat until queue is empty
    while queue: 
        vertex = queue.popleft() # 0:node - 1:level - 2:initial_node
        # If vertex is destionation return level
        if vertex[0] in destination_list:
            jumps_list.append(vertex[1])
        # Add to visited and append to queue
        for neighbour in dic_adj_graph[vertex[0]]: 
            if (neighbour, vertex[2]) not in visited: 
                visited.add((neighbour, vertex[2])) 
                queue.append((neighbour, vertex[1]+1, vertex[2])) 
    return jumps_list

In [50]:
# THIS FUNCTION IS ONLY FOR TIMING
def distance_time_our2():
    CxCy = []
    # Input category to make block ranking
    source_category = 'English_footballers'
    target_category = 'The_Football_League_players'

    C0 = list(categories_reduced.get(source_category)) #C0
    CI = set(categories_reduced.get(target_category)) #C1
    
    CxCy.extend(shortest_path_bfs_list_list(dic_adj, C0[:10], CI))

    print("#Results:", len(CxCy), "Median:", statistics.median(CxCy))
    return statistics.median(CxCy)

In [52]:
timeit.timeit(distance_time_our2, number = 1)

#Results: 48967 Median: 6


13.89341706358664

**We have made 3 test with 10 values of C0 and all values of C1 adn finally we are going to work with the implementation of Test 2**

## Build block ranking

In [34]:
def distance(C0, CI, n_nodes):
    CxCy = []

    C0 = list(categories_reduced.get(C0)) #C0
    CI = set(categories_reduced.get(CI)) #C1
    
    for s_value in C0[:n_nodes]:
        CxCy.extend(shortest_path_bfs_list(dic_adj, s_value, CI))

    return statistics.median(CxCy)

In [35]:
n_C0_nodes = 2
C0 = 'English_footballers'
block_ranking = [(0, C0)]
for cat in categories_reduced.keys():
    if cat != C0:
        block_ranking.append((distance(C0, cat, n_C0_nodes), cat))

In [36]:
block_ranking.sort()

In [37]:
pd_block_ranking = pd.DataFrame(block_ranking, columns=['Score','Category'])

In [129]:
pd_block_ranking

,Score,Category
0,0.0,English_footballers
1,5.0,Association_football_forwards
2,5.0,Association_football_goalkeepers
3,5.0,Association_football_midfielders
4,5.0,The_Football_League_players
5,6.0,American_film_actors
6,6.0,American_films
7,6.0,American_military_personnel_of_World_War_II
8,6.0,American_television_actors
9,6.0,Association_football_defenders


## Sort all the nodes of each category

In [38]:
#Info total graph
print(nx.info(Gdir))

Name: 
Type: DiGraph
Number of nodes: 461193
Number of edges: 2645247
Average in degree:   5.7357
Average out degree:   5.7357


### Testing

In [39]:
C0 = 'English_footballers'
C0_nodes = set(categories_reduced.get(C0)) #C0

#Compute subgraph
C0_sub = Gdir.subgraph(nodes=C0_nodes)

In [40]:
print(nx.info(C0_sub))

Name: 
Type: DiGraph
Number of nodes: 7538
Number of edges: 18233
Average in degree:   2.4188
Average out degree:   2.4188


In [41]:
for node in set(C0_sub.nodes):
    Gdir.node[node]['weight'] = len(list(C0_sub.in_edges(node)))

In [42]:
print(Gdir.node[132132]['weight'])

2


### Computing

Initiate or restart weight of all nodes:

In [53]:
#Compute node weights
    for node in set(Gdir.nodes):
        Gdir.node[node]['weight'] = 0

**Making STEP 1 2 3**

In [94]:
nodes_weight_computed = set()
for index_cat in range(len(pd_block_ranking)):
    print(pd_block_ranking.loc[index_cat][1], end="; ") # Just to know which categorie is being computed
    CI = pd_block_ranking.loc[index_cat][1] # Category name
    
    #Compute subgraph
    CI_sub = Gdir.subgraph(nodes=set(categories_reduced.get(CI))) # Make subgraph of the category

    #Compute node weights
    for node in set(CI_sub.nodes):
        aux_weight = 0
        
        # Look if the in edges of the node to compute have already weight and save aux
        for node_adj in Gdir.in_edges(node):
            if node_adj in nodes_weight_computed:
                aux_weight += Gdir.node[node_adj[0]]['weight']
            
        # Save weight from subgraph and from nodes with computed weight
        if node not in nodes_weight_computed:
            Gdir.node[node]['weight'] = len(list(CI_sub.in_edges(node))) + aux_weight
    
    nodes_weight_computed.union(set(CI_sub.nodes))

English_footballers; Association_football_forwards; Association_football_goalkeepers; Association_football_midfielders; The_Football_League_players; American_film_actors; American_films; American_military_personnel_of_World_War_II; American_television_actors; Association_football_defenders; British_films; English-language_films; People_from_New_York_City; Black-and-white_films; Debut_albums; English-language_albums; Harvard_University_alumni; Indian_films; Living_people; Major_League_Baseball_pitchers; Members_of_the_United_Kingdom_Parliament_for_English_constituencies; Place_of_birth_missing_(living_people); Rivers_of_Romania; Windows_games; Year_of_birth_missing; Year_of_birth_missing_(living_people); Year_of_death_missing; Asteroids_named_for_people; Main_Belt_asteroids; 

### Results

Just watch nodes with more than 1000 of weight

In [95]:
set_nodes = set(Gdir.nodes)
for node in list_nodes:
    if Gdir.node[node]['weight'] > 1000:
        print(node, Gdir.node[node]['weight'])

81952 3125
1400547 2627
1400548 3200
1400635 2304
870589 10555
1041937 1501


Making a list of list with the weight of the nodes of each category

In [114]:
list_scores = []
for index_cat in range(len(pd_block_ranking)):
    print(pd_block_ranking.loc[index_cat][1], end="; ") # Just to know which categorie is being computed
    CI = pd_block_ranking.loc[index_cat][1] # Category name
    list_scores.append([])
    #Compute subgraph
    for node in set(categories_reduced.get(CI)):
        list_scores[index_cat].append((Gdir.node[node]['weight'], node))


English_footballers; Association_football_forwards; Association_football_goalkeepers; Association_football_midfielders; The_Football_League_players; American_film_actors; American_films; American_military_personnel_of_World_War_II; American_television_actors; Association_football_defenders; British_films; English-language_films; People_from_New_York_City; Black-and-white_films; Debut_albums; English-language_albums; Harvard_University_alumni; Indian_films; Living_people; Major_League_Baseball_pitchers; Members_of_the_United_Kingdom_Parliament_for_English_constituencies; Place_of_birth_missing_(living_people); Rivers_of_Romania; Windows_games; Year_of_birth_missing; Year_of_birth_missing_(living_people); Year_of_death_missing; Asteroids_named_for_people; Main_Belt_asteroids; 

Order nodes by weight of each list of categories:

In [119]:
for index_cat in range(len(pd_block_ranking)):
    list_scores[index_cat].sort(key = lambda x: x[0], reverse=True)

Print ordered nodes by weight of each category:

In [126]:
for index_cat in range(len(pd_block_ranking)):
    print(pd_block_ranking.loc[index_cat][1], ':', *list_scores[index_cat][0:4])
          
#print('0', *list_scores[0][0:10], '\n1', list_scores[1][0:10], '\n2', list_scores[2][0:10])

English_footballers : (261, 82393) (163, 82365) (163, 81878) (133, 81847)
Association_football_forwards : (210, 89923) (163, 82365) (144, 81928) (131, 83512)
Association_football_goalkeepers : (3125, 81952) (71, 81854) (67, 82589) (63, 82414)
Association_football_midfielders : (261, 82393) (245, 89978) (163, 81878) (156, 81856)
The_Football_League_players : (261, 82393) (195, 82092) (163, 81878) (156, 81856)
American_film_actors : (869, 1179311) (617, 1061960) (536, 1184448) (497, 1062053)
American_films : (212, 572855) (67, 1063315) (45, 1247389) (40, 1063898)
American_military_personnel_of_World_War_II : (809, 1400483) (193, 1400484) (181, 1025555) (177, 1169029)
American_television_actors : (536, 1184448) (401, 1184788) (387, 1184217) (361, 1061971)
Association_football_defenders : (105, 81871) (87, 88804) (79, 85428) (77, 81787)
British_films : (1501, 1041937) (25, 1061229) (21, 1253706) (20, 1253703)
English-language_films : (67, 1063315) (64, 1061272) (58, 1061132) (58, 1061244)
